In [ ]:
import sys

rootpath = '/home/student/ROI/Spark/'
datapath = f'{rootpath}datasets/'
sys.path.append(rootpath)
from pyspark_helpers import *
sc, spark, conf = initspark()



In [27]:
import pandas as pd
import sklearn as sk
import numpy as np
from sklearn.datasets import load_iris
iris = load_iris()
print(iris.keys())
print(iris['feature_names'])
print(iris['target_names'])

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
['setosa' 'versicolor' 'virginica']


In [28]:
iris_features = iris['data']
print(type(iris_features))
iris_label = iris['target']
features = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

data1 = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= features + ['target'])
irisDF = spark.createDataFrame(data1)
display(irisDF)

irisPandas = irisDF.toPandas()


<class 'numpy.ndarray'>


,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
5,5.4,3.9,1.7,0.4,0.0
6,4.6,3.4,1.4,0.3,0.0
7,5.0,3.4,1.5,0.2,0.0
8,4.4,2.9,1.4,0.2,0.0
9,4.9,3.1,1.5,0.1,0.0


In [ ]:
from pyspark.ml.feature import VectorAssembler
vecAssembler = VectorAssembler(inputCols=features, outputCol="features")
dfML = vecAssembler.transform(irisDF)
display(dfML)


In [ ]:
from pyspark.ml.clustering import KMeans
import matplotlib.pyplot as plt
from pyspark.sql.functions import expr

CLUSTERS = 3

kmeans = KMeans(k=3, seed = 1)
model = kmeans.fit(dfML)
centroids = model.clusterCenters()
#print(centroids)

predictions = model.transform(dfML) 
#display(predictions)


x = predictions.select('prediction', 'target').collect()
print(x[0])
print('-->', tuple(x[0]))
print(x[0].prediction, x[0]['target'])

# good recipe to convert list of Row objects into list of tuples
print(list(map(tuple, x)))



In [ ]:
#df.withColumn('alias', expr('sql function'))

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import SQLTransformer
display(irisDF)
#irisDF2 = irisDF.withColumn('ratio', 'sepal_length/sepal_width')
sql = "SELECT *, sepal_length/sepal_width as sepal_ratio, petal_length/petal_width as petal_ratio FROM __THIS__"
sqlTransformer = SQLTransformer(statement = sql)
sqlModel = sqlTransformer.transform(irisDF)
display(sqlModel)


In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StandardScaler
display(irisDF.describe())

standardScaler = StandardScaler(inputCol='features', outputCol='scaled_features')
model = standardScaler.fit(dfML)
print(model.mean, model.std)
dfMLScaled = model.transform(dfML)
display(dfML)
display(dfMLScaled)


In [ ]:
#from pyspark.ml.linalg import Vectors
#from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.feature import RFormula

Rform = RFormula(formula = 'target~sepal_length + sepal_width + petal_length + petal_width').fit(irisDF).transform(irisDF).select('features', 'label')
display(Rform)


In [ ]:
df.where().withColumn().groupBy()
standardscaler.fit pass through to figure out the mean std
.transform apply the calculation to each row

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import SQLTransformer
from pyspark.ml.feature import RFormula

quantile = QuantileDiscretizer(inputCol='sepal_width', outputCol='sepal_width_bucket', numBuckets = 2).fit(irisDF).transform(irisDF)

display(quantile)

# # d = {'inputCol':'sepal_length', 'outputCol'='sepal_length_bucket', 'numBuckets' = 2}
# # d1 = dict(inputCol='sepal_length', outputCol='sepal_length_bucket', numBuckets = 2)
# # quantile1 = QuantileDiscretizer(**d)

quantile1 = QuantileDiscretizer(inputCol='sepal_length', outputCol='sepal_length_bucket', numBuckets = 2)
quantile2 = QuantileDiscretizer(inputCol='sepal_width', outputCol='sepal_width_bucket', numBuckets = 2)
quantile3 = QuantileDiscretizer(inputCol='petal_length', outputCol='petal_length_bucket', numBuckets = 2)
quantile4 = QuantileDiscretizer(inputCol='petal_width', outputCol='petal_width_bucket', numBuckets = 2)


sql = "SELECT *, sepal_length * sepal_length_bucket as sl FROM __THIS__"
sqlTransformer = SQLTransformer(statement = sql)


Rform = RFormula(formula = 'target~sepal_length + sepal_width + petal_length + petal_width')

sql2 = "SELECT features, label FROM __THIS__"
sqlTransformer2 = SQLTransformer(statement = sql2)

stages = [quantile1, quantile2, sqlTransformer, quantile3, quantile4, Rform, sqlTransformer2]

pipeline = Pipeline(stages = stages)

dfPipeModel = pipeline.fit(irisDF)
dfPipeModel.write().overwrite().save('irisPipeline3')

dfPipe = dfPipeModel.transform(irisDF)
dfML = dfPipe.select('label', 'features')
display(dfPipe2)

In [29]:
from pyspark.ml.pipeline import PipelineModel
dfPipeModel2 = PipelineModel.load('irisPipeline3')
dfML = dfPipeModel2.transform(irisDF)
display(dfML)



,features,label
0,"[5.1, 3.5, 1.4, 0.2]",0.0
1,"[4.9, 3.0, 1.4, 0.2]",0.0
2,"[4.7, 3.2, 1.3, 0.2]",0.0
3,"[4.6, 3.1, 1.5, 0.2]",0.0
4,"[5.0, 3.6, 1.4, 0.2]",0.0
5,"[5.4, 3.9, 1.7, 0.4]",0.0
6,"[4.6, 3.4, 1.4, 0.3]",0.0
7,"[5.0, 3.4, 1.5, 0.2]",0.0
8,"[4.4, 2.9, 1.4, 0.2]",0.0
9,"[4.9, 3.1, 1.5, 0.1]",0.0


In [ ]:
def predict_one(sl, sw, pl, pw):
    df = spark.createDataFrame([{'sepal_length':sl, 'sepal_width':sw, 'petal_length':pl, 'petal_width':pw, 'label':0}])
    dfML = dfPipeModel2.transform(df)
    predict = someClassModel.transform(dfML)
    return predict

display(predict_one(5.1, 3.5,1.4,0.2))


In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import MaxAbsScaler
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import SQLTransformer
from pyspark.ml.feature import PCA
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.feature import RFormula

#    , 'SQL' : (SQLTransformer, {'statement':sql})
#    , 'RFormula' : (RFormula, {'formula':'target~sepal_length + sepal_width + petal_length + petal_width'})
#    , 'quantile' : (QuantileDiscretizer, {'inputOutput':True, 'numBuckets':2})


scalers = {
      'standard' : (StandardScaler,{})
    , 'maxAbs' : (MaxAbsScaler,{})
    , 'minMax' : (MinMaxScaler,{})
    , 'PCA' : (PCA, {'k':2})
}

for name, scalerClass in scalers.items():
    print (name)
    c, p = scalerClass
    p.update({'inputCol':'features', 'outputCol':name+'_features'})
    print (p)
    scaler = c(**p)

    model = scaler.fit(dfML).transform(dfML)
    display(model)


In [43]:
from pyspark.sql.functions import mean, expr
from pyspark.ml.feature import Imputer

df = spark.createDataFrame([{"col1":1, "col2":"A", "col3":None, "col4":None}
                            , {"col1":2, "col2":None, "col3":None, "col4":4.0}
                           , {"col1":3, "col2":None, "col3":2, "col4":10.0}
                           , {"col1":4, "col2":"D", "col3":4, "col4":6.0}]
                          , schema = "col1:int, col2:string, col3:int, col4:float")
display(df)

display(df.fillna(0))
m = df.select(mean(df.col3).alias('mean')).collect()[0]['mean']
m2 = df.select(mean(df.col4).alias('mean')).collect()[0]['mean']
print(m)
display(df.fillna({"col2":"x", "col3":m, "col4":m2}))

from pyspark.ml.feature import Imputer

cols = ['col3', 'col4']
# imputer = Imputer(inputCols = cols, outputCols = [f'{c}_imputed' for c in cols], strategy = 'mean')
# display(imputer.fit(df).transform(df))

df = df.withColumn('col3', expr('cast(col3 as float)'))
display(df)
imputer = Imputer(inputCols = cols, outputCols = [f'{c}_imputed' for c in cols],
                  strategy = 'mean')
display(imputer.fit(df).transform(df))



,col1,col2,col3,col4
0,1,A,NaN,NaN
1,2,None,NaN,4.0
2,3,None,2.0,10.0
3,4,D,4.0,6.0


,col1,col2,col3,col4
0,1,A,0,0.0
1,2,None,0,4.0
2,3,None,2,10.0
3,4,D,4,6.0


3.0


,col1,col2,col3,col4
0,1,A,3,6.666667
1,2,x,3,4.000000
2,3,x,2,10.000000
3,4,D,4,6.000000


,col1,col2,col3,col4
0,1,A,NaN,NaN
1,2,None,NaN,4.0
2,3,None,2.0,10.0
3,4,D,4.0,6.0


,col1,col2,col3,col4,col3_imputed,col4_imputed
0,1,A,NaN,NaN,3.0,6.666667
1,2,None,NaN,4.0,3.0,4.000000
2,3,None,2.0,10.0,2.0,10.000000
3,4,D,4.0,6.0,4.0,6.000000


In [31]:
import pandas as pd
import sklearn as sk
import numpy as np
from sklearn.datasets import load_iris
iris = load_iris()
iris_features = iris['data']
iris_label = iris['target']
features = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

data1 = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= features + ['target'])
irisDF = spark.createDataFrame(data1)

from pyspark.ml.feature import PCA, VectorAssembler, VectorDisassembler
from pyspark.ml.pipeline import Pipeline

vec = VectorAssembler(inputCols = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], outputCol = 'PCA')
pca = PCA(k = 3, inputCol = 'PCA', outputCol = 'PCA_components')
dis = VectorDisassembler(inputCol = 'PCA_components')
pipeline = Pipeline(stages = [vec, pca, dis])

dfPipeModel = pipeline.fit(irisDF)
dfPipe = dfPipeModel.transform(irisDF)
dfPipe.show()



,features,label
0,"[5.1, 3.5, 1.4, 0.2]",0.0
1,"[4.9, 3.0, 1.4, 0.2]",0.0
2,"[4.7, 3.2, 1.3, 0.2]",0.0
3,"[4.6, 3.1, 1.5, 0.2]",0.0
4,"[5.0, 3.6, 1.4, 0.2]",0.0
5,"[5.4, 3.9, 1.7, 0.4]",0.0
6,"[4.6, 3.4, 1.4, 0.3]",0.0
7,"[5.0, 3.4, 1.5, 0.2]",0.0
8,"[4.4, 2.9, 1.4, 0.2]",0.0
9,"[4.9, 3.1, 1.5, 0.1]",0.0


ImportError: cannot import name 'VectorDisassembler' from 'pyspark.ml.feature' (/home/student/anaconda3/lib/python3.7/site-packages/pyspark/ml/feature.py)